In [1]:
import numpy as np
import pandas as pd
import spacy
import string
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model
import tensorflow_datasets as tfds
import random
from sklearn.model_selection import train_test_split
from tensorflow.python.client import device_lib

In C:\Users\renat\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\renat\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\renat\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\renat\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\renat\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

In [2]:
data = pd.read_json('tcc1.json')

In [7]:
!python -m spacy download pt_core_news_sm
!pip install -U spacy[cuda92]

[!] Skipping model package dependencies and setting `--no-deps`. You don't seem
to have the spaCy package itself installed (maybe because you've built from
source?), so installing the model dependencies would cause spaCy to be
downloaded, which probably isn't what you want. If the model package has other
dependencies, you'll have to install them manually.
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.3.0-cp37-none-any.whl size=19557503 sha256=aa537d03535a28f07a3b7cfea06f7f16c2d5fb20420696572b2a5e38e94b8e29
  Stored in directory: C:\Users\renat\AppData\Local\Temp\pip-ephem-wheel-cache-1efp028h\wheels\f6\43\f6\949b5e2b48df37aa3ed81fc3f4af4154afa276518cc004b990
Successfully built pt-core-news-sm
[+] Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


In [11]:
nlp = spacy.load("pt_core_news_sm")

OSError: [E050] Can't find model 'pt_core_news_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [88]:
stopwords = spacy.lang.pt.STOP_WORDS
pontos = string.punctuation

In [89]:
def frase_limpa(texto):
    palavra = []
    texto = texto.lower()
    texto = nlp(texto)
    for p in texto:
        palavra.append(p.text)
    texto = [p for p in palavra if p not in stopwords or p not in pontos]
    texto = ' '.join([i for i in texto])
    return texto

In [90]:
noticia_frase = []
for i in range(len(data.texto)):
    noticia_frase.append(frase_limpa(data.texto[i]))

In [91]:
titulo_frase = []
for i in range(len(data.título)):
    titulo_frase.append(frase_limpa(data.título[i]))

In [102]:
tokenizador = tfds.features.text.SubwordTextEncoder.build_from_corpus(noticia_frase, target_vocab_size=2**10)

In [103]:
tam_max = max([len(frase)] for frase in frase_input)
tam_max = tam_max[0]

In [104]:
tam_max

3022

In [105]:
frase_input = tf.keras.preprocessing.sequence.pad_sequences(frase_input, value = 0, padding='post', maxlen=tam_max)

In [106]:
titulo_input = [tokenizador.encode(titulo) for titulo in titulo_frase]

In [107]:
titulo_max = max([len(titulo)] for titulo in titulo_input)
titulo_max = titulo_max[0]

In [108]:
titulo_max

56

In [109]:
titulo_input = tf.keras.preprocessing.sequence.pad_sequences(titulo_input, value = 0, padding='post', maxlen=tam_max)

In [118]:
vocab_size = tokenizador.vocab_size
emb_dim = 50
nb_filter = 25
ffn_units = 128
dropout_rate = 0.2
nb_epochs = 100

In [124]:
encoder_input_data = np.zeros(
    (len(frase_input), tam_max, vocab_size),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(frase_input), titulo_max, vocab_size),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(titulo_input), titulo_max, vocab_size),
    dtype='float32')

In [125]:
encoder_input = Input(shape=(None, tam_max))
encoder = LSTM(emb_dim, return_state=True)
encoder_output, state_h, state_c = encoder(encoder_input)
encoder_state = [state_h, state_c]

In [126]:
decoder_input = Input(shape=(None, titulo_max))
decoder_lstm = LSTM(emb_dim, return_sequences=True, return_state=True)
decoder_output, _, _ = decoder_lstm(decoder_input, initial_state=encoder_state)
decoder_dense = Dense(titulo_max, activation='softmax')
decoder_output = decoder_dense(decoder_output)

In [127]:
decoder_target_data = np.zeros((len(titulo_input), titulo_max), dtype='float32')

In [128]:
model = Model([encoder_input, decoder_input], decoder_output)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, None, 3022)   0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, None, 56)     0                                            
__________________________________________________________________________________________________
lstm_7 (LSTM)                   [(None, 50), (None,  614600      input_7[0][0]                    
__________________________________________________________________________________________________
lstm_8 (LSTM)                   [(None, None, 50), ( 21400       input_8[0][0]                    
                                                                 lstm_7[0][1]               

In [129]:
model.fit([encoder_input_data, decoder_target_data], batch_size=64, epochs=nb_epochs, validation_split=0.2)

ValueError: Error when checking input: expected input_7 to have shape (None, 3022) but got array with shape (3022, 1031)